In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


## Load the training data

In [4]:
train_data = pd.read_csv("train.csv")
train_data['Survived']

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

## Load the test data

In [5]:
test_data = pd.read_csv("test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## Check the quality of data

In [6]:
train_data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

## Remove data that is not useful

In [7]:
train_data = train_data.drop(columns=['Cabin'], axis=1)
train_data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         2
dtype: int64

## Replace missing data

In [8]:
#Finding the mean of the Age column having NaN
mean_value=train_data['Age'].mean()
print(mean_value)

train_data['Age'].fillna(value=mean_value, inplace=True)
train_data.isna().sum()

#Finding the mode of the Embarked column having NaN
embarked_mode = train_data['Embarked'].mode()[0]
train_data['Embarked'].fillna(value=embarked_mode, inplace=True)
train_data.isna().sum()

29.69911764705882


PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

## Predict Survival for other passengers

In [25]:
from sklearn.ensemble import RandomForestClassifier

# extract the dependent variable. note: this is already in the correct format
y = train_data["Survived"]

features = ["Pclass", "Sex", "SibSp", "Parch"]
# convert categorical data into dummy data for processing. I dont think this requires more advanced encoding like label or one hot because
# the only categorical value in here truly is Sex which is male and female. Im kind of curious if creating two dummy columns will hurt the model.
X = pd.get_dummies(train_data[features])
# important to note here that test data, by default, does not have a Survived column. This is nice for the exercise but not realistic. In reality, we would need to separate out our train and test data sets to compare
X_test = pd.get_dummies(test_data[features])


# max depth is 5 because thats how many columns we are assessing
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
did_survive = train_data.loc[train_data['Survived'] == 1]["Survived"]
perc_survived = (did_survive.sum()/len(train_data["Survived"])) * 100
print(f"""The random forest classifier model predicts that {round(perc_survived, 2)}% of people
in the test set would have survived the titanic based on their Pclass, Sex, SibSp and Parch""")
# make sure there is no missing data

The random forest classifier model predicts that 38.38% of people
in the test set would have survived the titanic based on their Pclass, Sex, SibSp and Parch
